# Reddit Resume image Scraper By visiting All available Result 

In [1]:
import os
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urlparse
import time

chrome_options = Options()
chrome_options.add_argument("--disable-gpu") 

driver = webdriver.Chrome(options=chrome_options)

search_url = "https://www.reddit.com/r/resumes/search/?q=computer+science+resume+india"
driver.get(search_url)

# Wait for the search results to load
WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.TAG_NAME, "reddit-feed"))
)

# Create a folder to save images if it doesn't exist
if not os.path.exists('downloaded_images'):
    os.makedirs('downloaded_images')

# Initialize counter for downloaded images and a set to track visited URLs
image_count = 1  

supported_extensions = ['.jpg', '.jpeg', '.png', '.webp']

# Keep track of processed URLs to avoid duplicates
processed_urls = set()

# Main scraping loop: iterating over each link in search results
while True:
    
    reddit_feed = driver.find_element(By.TAG_NAME, "reddit-feed")
    aria_links = reddit_feed.find_elements(By.CSS_SELECTOR, "a[aria-label]")
    
    # Process each aria link found
    for link in aria_links:
        post_url = link.get_attribute("href")
        
        # Skip the link if it's already been processed
        if not post_url or post_url in processed_urls:
            continue

        # Add the URL to the set of processed URLs
        processed_urls.add(post_url)

        # Open the post in a new tab
        driver.execute_script("window.open(arguments[0], '_blank');", post_url)
        driver.switch_to.window(driver.window_handles[-1])

        # Wait for the post page to load and image to be visible
        try:
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'media-lightbox-img'))
            )

            # Find the first image div within media-lightbox-img
            media_divs = driver.find_elements(By.CLASS_NAME, 'media-lightbox-img')
            if media_divs:
                img = media_divs[0].find_element(By.TAG_NAME, 'img')
                img_url = img.get_attribute('src')

                if img_url:
                    print(f"Found image URL: {img_url}")
                    
                    # Clean URL for file extension
                    parsed_url = urlparse(img_url)
                    base_url = parsed_url.path  
                    
                    # Check if URL ends with a supported extension and download
                    if any(base_url.endswith(ext) for ext in supported_extensions):
                        image_name = f"image{image_count}.{base_url.split('.')[-1]}"
                        image_path = os.path.join('downloaded_images', image_name)

                        # Download the image
                        try:
                            response = requests.get(img_url, timeout=10)
                            response.raise_for_status()
                            with open(image_path, 'wb') as file:
                                file.write(response.content)
                            print(f"Downloaded image: {image_path}")
                            image_count += 1
                        except requests.exceptions.RequestException as e:
                            print(f"Failed to download image from {img_url}. Error: {e}")
                    else:
                        print(f"Skipping unsupported image URL: {img_url}")
            else:
                print("No media divs with images found in this post.")

            # Close the tab after processing
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        
        except Exception as e:
            print(f"Error processing link: {post_url}. Error: {e}")
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        
        # Pause between iterations
        time.sleep(1)

    # If no new links are found, end the loop
    if not aria_links:
        break

print(f"Total images downloaded: {image_count - 1}")

driver.quit()


The chromedriver version (129.0.6668.89) detected in PATH at chromedriver.EXE might not be compatible with the detected chrome version (130.0.6723.117); currently, chromedriver 130.0.6723.116 is recommended for chrome 130.*, so it is advised to delete the driver in PATH and retry


Found image URL: https://preview.redd.it/this-is-my-resume-i-am-computer-science-student-graduating-v0-sfzfhqowtg5c1.jpg?width=640&crop=smart&auto=webp&s=83188a307ae74999b5f0dfac9e5feb4a5e021403
Downloaded image: downloaded_images\image1.jpg
Found image URL: https://preview.redd.it/x9q8lag49sq51.jpg?width=640&crop=smart&auto=webp&s=2d796430f41b69dbb68099c921df26bcb685fdca
Downloaded image: downloaded_images\image2.jpg
Found image URL: https://preview.redd.it/0-yoe-unemployed-recent-cybersecurity-graduate-applying-for-v0-31oewpqo9qyd1.jpeg?width=640&crop=smart&auto=webp&s=999b182f992736ef84eae6dac162998683e58dbf
Downloaded image: downloaded_images\image3.jpeg
Found image URL: https://preview.redd.it/mqxmcmx0xk971.jpg?width=640&crop=smart&auto=webp&s=c5a449f3e38cdc399cc46f56e0ae1dd286ddbe37
Downloaded image: downloaded_images\image4.jpg
Found image URL: https://preview.redd.it/1-yoe-data-science-intern-rs-technologies-india-v0-j4uri2eb52yd1.png?width=640&crop=smart&auto=webp&s=092ebf76f8

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=130.0.6723.117)
Stacktrace:
	GetHandleVerifier [0x01137143+25587]
	(No symbol) [0x010CA2E4]
	(No symbol) [0x00FC2113]
	(No symbol) [0x00F9E23B]
	(No symbol) [0x0103179F]
	(No symbol) [0x01044CB9]
	(No symbol) [0x0102A936]
	(No symbol) [0x00FFBA73]
	(No symbol) [0x00FFC4CD]
	GetHandleVerifier [0x01414C63+3030803]
	GetHandleVerifier [0x01466B99+3366473]
	GetHandleVerifier [0x011C95F2+624802]
	GetHandleVerifier [0x011D0E6C+655644]
	(No symbol) [0x010D2C9D]
	(No symbol) [0x010CFD68]
	(No symbol) [0x010CFF05]
	(No symbol) [0x010C2336]
	BaseThreadInitThunk [0x766FFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x7765809E+286]
	RtlGetAppContainerNamedObjectPath [0x7765806E+238]


# Extract Text From Images  

In [ ]:
import pytesseract
from PIL import Image
import re

pytesseract.pytesseract.tesseract_cmd = r"C:/Program Files/Tesseract-OCR/tesseract.exe"  

image_path = "D:/Machine Learning/WEB_Scraping/new2/downloaded_images/image4.jpg"  
img = Image.open(image_path)

# Extract text using pytesseract
extracted_text = pytesseract.image_to_string(img)

# Define sections we want to extract
sections = {
    "Contact Info": r"NAME SURNAME[\s\S]+?(?=SKILLS)",
    "Skills": r"SKILLS[\s\S]+?(?=PROJECTS)",
    "Projects": r"PROJECTS[\s\S]+?(?=WORK EXPERIENCE)",
    "Work Experience": r"WORK EXPERIENCE[\s\S]+?(?=EDUCATION)",
    "Education": r"EDUCATION[\s\S]+"
}

# Extract each section based on patterns
structured_text = {}
for section, pattern in sections.items():
    match = re.search(pattern, extracted_text, re.IGNORECASE)
    if match:
        structured_text[section] = match.group().strip()

# Display structured text
for section, content in structured_text.items():
    print(f"{section}:\n{content}\n{'-' * 50}\n")


Education:
Education
ABC University

Programming Languages: Java, JavaScript, Python, SQL,
Web Teclnologies & Databases: ITML, CSS, Sas/SCSS, MySQL, MongoDB, PostgreSQL.
Framoworks & Libraries: React, Reus, React Router, Nodes, Express, mongoose, Semantic UI, Bootstrap
Developer Tools: Git, GitKrakeu, GitHlub, Postman, npm, Webpack, Babel, WekStorm, VS Code, Eeipss, PyCharm

Projects

Project 1

+ Engineered a robotic ear that
‘object recognition using tensor”

+ Programmed autonomous navigation ofthe robot ad inplemented spooch synthenis

Mer 2021 ~ Jun 2021
provide navigation ail forthe bd and visually inpaired by perforiing Rea

«Spak and Python
+ Achieved object recognition of prevalent items with 905 accuracy’ by implementing coco dataset sing tensortlow ite
+ Technologies Used: Raspberry Pi, Python, TensorFlow, OpeuCV, NuaaPy

Project 2 | Wetsite Li May 2021 ~ Jun 2021
+ Developed an e-commerce platform along with an onder processing workflow and Content Management System (CMS)
+